In [ ]:
# TODO: I think this could all be done more efficiently using feature-engine, e.g. WindowFeatures, etc. 
#   Check this out once I've done it his way

# TODO: Check out paralell processing using: joblib, Ray, Dask. For loops are slow in python so this is worth doing for
#   bigger jobs

In [2]:
import pandas as pd

ts_data = pd.read_parquet("../data/transformed/ts_data_2022_01.parquet")
ts_data.head()

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,11,4
1,2022-01-01 01:00:00,15,4
2,2022-01-01 02:00:00,26,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,9,4


In [3]:
# Just slice for a single location at first.
# Not sure we need the reset_index here (see next cell)
ts_data_one_location = ts_data.loc[ts_data['pickup_location_id'] == 43, :].reset_index(drop=True)
ts_data_one_location.head()

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,97,43
1,2022-01-01 01:00:00,60,43
2,2022-01-01 02:00:00,22,43
3,2022-01-01 03:00:00,8,43
4,2022-01-01 04:00:00,6,43


In [4]:
ts_data.loc[ts_data['pickup_location_id'] == 43, :]

,pickup_hour,rides,pickup_location_id
7440,2022-01-01 00:00:00,97,43
7441,2022-01-01 01:00:00,60,43
7442,2022-01-01 02:00:00,22,43
7443,2022-01-01 03:00:00,8,43
7444,2022-01-01 04:00:00,6,43
...,...,...,...
8179,2022-01-31 19:00:00,61,43
8180,2022-01-31 20:00:00,73,43
8181,2022-01-31 21:00:00,33,43
8182,2022-01-31 22:00:00,21,43


- We'll obtain the features for training by choosing a window size, e.g. 24 hrs, and using these values to predict the demand for the next hour. 
- To produce the entire training set, we write a function that slides across the dataset and gets the training start and stop indices, plus the index for the associated target. 

In [5]:
# Function to obtain training indices
def get_cutoff_indices(
    data: pd.DataFrame,
    n_features: int, 
    step_size: int) -> list:
    
    stop_position = len(data) - 1
    
    # start first slice at index 0
    subseq_first_idx = 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features + 1
    indices = []
    
    while subseq_last_idx <= stop_position:
        indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))
        
        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size
        
    return indices
    

In [7]:
# Test function
n_features = 24
step_size = 1

indices = get_cutoff_indices(
    data=ts_data_one_location,
    n_features=n_features,
    step_size=step_size
)

indices[:5]

[(0, 24, 25), (1, 25, 26), (2, 26, 27), (3, 27, 28), (4, 28, 29)]

In [16]:
# generate training data
import numpy as np

n_examples = len(indices)

x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=n_examples, dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]]['rides']
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

In [21]:
# Check output is what we want. 
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(719, 24)
x=array([[ 97.,  60.,  22., ...,  16.,  18.,   6.],
       [ 60.,  22.,   8., ...,  18.,   6.,   3.],
       [ 22.,   8.,   6., ...,   6.,   3.,   1.],
       ...,
       [ 28.,  16.,  13., ..., 102.,  66.,  61.],
       [ 16.,  13.,   8., ...,  66.,  61.,  73.],
       [ 13.,   8.,   1., ...,  61.,  73.,  33.]], dtype=float32)
pickup_hours[:5]=[Timestamp('2022-01-02 00:00:00'), Timestamp('2022-01-02 01:00:00'), Timestamp('2022-01-02 02:00:00'), Timestamp('2022-01-02 03:00:00'), Timestamp('2022-01-02 04:00:00')]


In [22]:
# Now convert to a dataframe
features_one_location = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)

features_one_location.head()

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,97.0,60.0,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,...,70.0,94.0,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0
1,60.0,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,...,94.0,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0
2,22.0,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,...,87.0,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0
3,8.0,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,35.0,...,73.0,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0,1.0
4,6.0,5.0,3.0,10.0,7.0,19.0,24.0,39.0,35.0,77.0,...,34.0,32.0,22.0,16.0,18.0,6.0,3.0,1.0,1.0,0.0


In [23]:
# Convert target to dataframe
targets_one_location = pd.DataFrame(
    y,
    columns=['target_rides_next_hour']
)

targets_one_location.head()

,target_rides_next_hour
0,3.0
1,1.0
2,1.0
3,0.0
4,0.0


In [27]:
# Function to do this for all locations
from tqdm import tqdm

def transform_ts_into_features_and_targets(
    ts_data: pd.DataFrame,
    n_features: int,
    step_size: int  
) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format to (features, target) for training
    """
    assert set(ts_data.columns) == {'pickup_hour', 'rides', 'pickup_location_id'}
    
    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()
    
    for location_id in tqdm(location_ids):
        
        # keep only data for this location_id
        ts_data_one_location = ts_data.loc[
            ts_data['pickup_location_id'] == location_id,
            ['pickup_hour', 'rides']
        ]
        
        # pre-compute cutoff indices to split dataframe rows
        indices = get_cutoff_indices(
            ts_data_one_location,
            n_features,
            step_size
        )
        
        # Slice and transpose data into np arrays
        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
        y = np.ndarray(shape=n_examples, dtype=np.float32)
        pickup_hours = []

        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]]['rides']
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])
            
        # features np -> pd
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
            )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id
        
        # target np -> pd
        targets_one_location = pd.DataFrame(
            y,
            columns=['target_rides_next_hour']
            )
        
        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])
        
    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True) 
    
    return features, targets['target_rides_next_hour']        

In [28]:
features, targets = transform_ts_into_features_and_targets(
    ts_data,
    n_features=24*7*1, # One week of history
    step_size=24 # to speed things up
)

print(f"{features.shape=}")
print(f"{targets.shape=}")

100%|██████████| 257/257 [00:00<00:00, 332.24it/s]

features.shape=(6168, 170)
targets.shape=(6168,)
